## PROJECT

Deliverables:
    Notebook
    Documentation of the model
    Test that everything works well
Goals:
    Compare the environmental performance of 2 biobased insulation materials contrasted to a reference fossil insulation      material, applied     to buildings throught the prodution and end-of-life phases of the product.
Scope:
    The environmental performance is assessed through the climate change (CO2, Ch4, N2O), water use, and land use (area occupied) impacts
    LCIA method: 
    The assessment is performed for 50 years of lifetime of the buildings
    The EoL choices involve incineration, landfill, composting, and mulching (we are going to consider 2)
    Temporal scope:
    
Tools
    Github
    Temporalis
    Excel

  *How to consider the soil carbon sequestration/release/lost effect
  For croplands is shor, for forests is large


Prerequisites:
    Python: nbdev and ipytest
Your environment must contain nbdev and nbdev_install_hooks
For this project:
Open conda terminal 
conda activate bw25
    This env already contains nbdev 

In [110]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
#pip install dynamic_characterization
from dynamic_characterization.ipcc_ar6.radiative_forcing import characterize_co2, characterize_ch4, characterize_n2o, characterize_co2_uptake
import copy
import bw2io as bi 
import bw2data as bd
import bw2calc as bc
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

In [46]:
list(bd.projects)

[Project: default,
 Project: SMFs,
 Project: bw25_intro,
 Project: bw25,
 Project: carbon fiber 2,
 Project: School,
 Project: Parametric LCA,
 Project: parametric-LCA,
 Project: SCO_LCA_v0,
 Project: Temporalis example project,
 Project: bw_temporalis example,
 Project: odym_case,
 Project: Buildings Insulation,
 Project: odym,
 Project: My BW25workgroup]

In [47]:
#Load ecoinvent and biosphere databases
#Either import from ecoinvent or load it from a file saved in your computer - Here we load from a filed saved from a previous project
bi.backup.restore_project_directory(
    fp=r"C:\Users\andraded\Documents\GitHub\brightway2-project-bw25_workgroup-backup14-November-2024-03-02PM.tar.gz",
    project_name="Buildings Insulation",
    overwrite_existing=True
)

Restoring project backup archive - this could take a few minutes...
Restored project: Buildings Insulation


'Buildings Insulation'

In [48]:
#set the project to work on
bd.projects.set_current('Buildings Insulation')

In [62]:
#check the databases in the project, be sure that the databases retrieved are now in the project
bd.databases

Databases dictionary with 2 object(s):
	biosphere
	ecoinvent-3.10-cutoff

In [50]:
#import your LCI
imp = bi.ExcelImporter(r"C:\Users\andraded\Documents\GitHub\LCI_Bw_form.xlsx")
imp.apply_strategies()

Extracted 1 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 24.52 seconds


In [51]:
#match the LCI with the ecoinvent data
imp.match_database('ecoinvent-3.10-cutoff', fields=('name', 'unit', 'location', 'reference product'))
imp.statistics()

Applying strategy: link_iterable_by_fields
4 datasets
	15 exchanges
	Links to the following databases:
		ecoinvent-3.10-cutoff (9 exchanges)
		Insulators (4 exchanges)
		biosphere (2 exchanges)
	0 unlinked exchanges (0 types)
		


(4, 15, 0, 0)

In [41]:
#if any unlik process appear, check where and solve it
#imp.write_excel()

Wrote matching file to:
C:\Users\andraded\AppData\Local\pylca\Brightway3\Buildings-Insulation.2ffa2648\output\db-matching-Insulators.xlsx


WindowsPath('C:/Users/andraded/AppData/Local/pylca/Brightway3/Buildings-Insulation.2ffa2648/output/db-matching-Insulators.xlsx')

In [142]:
imp.write_database()

12:32:20 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1013.79it/s]


12:32:20 [info     ] Vacuuming database            
Created database: Insulators


We have to create a db with relative years, for which we set here the (arbitrary) year 0 as the year of production. The definition of year 0 is irrelevant, the analysis is performed with absolute dates, the important task is to set correct relative timesteps.

In [128]:
yr_prod = 0
new_db_name = 'time'

First we need to find the code of the unit process we want to include in the background of the temporal data in this case for the EPS insulators: polystyrene foam slab production, treatment of waste polystyrene, sanitary landfill

In [55]:
db_eco = bd.Database('ecoinvent-3.10-cutoff')

In [68]:
#define the activity to temporlize
polystyrene= [act for act in db_eco if 'polystyrene foam slab production' in act['name']
                        and 'polystyrene foam slab' in act['reference product']
                        and act['location']=='RER'
              ][0]


In [73]:
#Look at the activity code
polystyrene.as_dict()

{'comment': 'Combination of material and processing module. EPS foam slab has a density of 30 kg/m3 and a thermal conductivity of 0.035-0.04 W/mK.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality guidelines for version 2. It may have been subject to central changes described in the ecoinvent version 3 change report (http://www.ecoinvent.org/database/ecoinvent-version-3/reports-of-changes/), and the results of the central updates were reviewed extensively. The changes added e.g. consistent water flows and other information throughout the database. The documentation of this dataset can be found in the ecoinvent reports of version 2, which are still available via the ecoinvent website. The change

In [87]:
#check exchanges within the activity
for exc in polystyrene.exchanges():
    print(exc) 

Exchange: 1.0 kilogram 'polystyrene foam slab production' (kilogram, RER, None) to 'polystyrene foam slab production' (kilogram, RER, None)>
Exchange: 1.0 kilogram 'polymer foaming' (kilogram, RER, None) to 'polystyrene foam slab production' (kilogram, RER, None)>
Exchange: 1.04 kilogram 'market for polystyrene, expandable' (kilogram, GLO, None) to 'polystyrene foam slab production' (kilogram, RER, None)>


In [88]:
#Look at codes of the exchanges in the activity
#polymer_foaming=[act for act in db_eco if 'polymer foaming' in act['name']
    #                    and 'polymer foaming' in act['reference product']
     #                   and act['location']=='RER'
      #        ][0]
#polymer_foaming.as_dict()
#market_for_polystyrene=[act for act in db_eco if 'market for polystyrene, expandable' in act['name']
    #                    and 'polystyrene, expandable' in act['reference product']
     #                   and act['location']=='GLO'
      #        ][0]
#market_for_polystyrene.as_dict()

In [101]:
#define activity 
polystyrene_act= {
'polystyrene foam slab production': 
        ('ecoinvent-3.10-cutoff', 'de2b60a30ff7830518ec8873f647d26e')
}

In [119]:
#define activity upstream
polystyrene_upstream= {
'polymer foaming': 
        ('ecoinvent-3.10-cutoff', '0e31d1fcab5f7b6c142245fdde30dd9e'),
'market_for_polystyrene': # 'sawlog and veneer log, softwood, measured as solid wood under bark'
        ('ecoinvent-3.10-cutoff', '31c71922d2a8871fa590470a67c8193f')
}

In [124]:
polystyrene_upstream

{'polymer foaming': ('ecoinvent-3.10-cutoff',
  '0e31d1fcab5f7b6c142245fdde30dd9e'),
 'market_for_polystyrene': ('ecoinvent-3.10-cutoff',
  '31c71922d2a8871fa590470a67c8193f')}

In [89]:
polymer_foaming=[act for act in db_eco if 'polymer foaming' in act['name']
                        and 'polymer foaming' in act['reference product']
                        and act['location']=='RER'
              ][0]

In [92]:
polymer_foaming.as_dict()

{'comment': '1 kg of this process equals 1 kg of expanded plastics (usually polystyrene).\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality guidelines for version 2. It may have been subject to central changes described in the ecoinvent version 3 change report (http://www.ecoinvent.org/database/ecoinvent-version-3/reports-of-changes/), and the results of the central updates were reviewed extensively. The changes added e.g. consistent water flows and other information throughout the database. The documentation of this dataset can be found in the ecoinvent reports of version 2, which are still available via the ecoinvent website. The change report linked above covers all central changes that were

In [91]:
market_for_polystyrene=[act for act in db_eco if 'market for polystyrene, expandable' in act['name']
                        and 'polystyrene, expandable' in act['reference product']
                        and act['location']=='GLO'
              ][0]

In [94]:
market_for_polystyrene.as_dict()

{'comment': "This is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for transport to the consumer and for the losses during that process, when relevant.\nThis is the market for  'polystyrene, expandable', in the Global geography.\nIn this market, expert judgement was used to develop product specific transport distance estimations.\n'polystyrene, expandable', is a plastic product of fossil origin, it is not biodegradable and it is a thermoplastic material. This product consists of 100% virgin material with no content of recycled material. The product is used in the following applications and sectors: thermal insulation in buildings, road construction, sound insulation, packaging, food packaging to maintain the temperature o

In [74]:
polystyrene_treatment=[act for act in db_eco if 'treatment of waste polystyrene' in act['name']
                        and 'sanitary landfill' in act ['name']
                        and 'waste polystyrene' in act['reference product']
                        and act['location']=='CH'
              ][0]

In [129]:
#change the original to a temporal one

def change_polystyrene_act(activity_code):
    old_db_name='ecoinvent-3.10-cutoff'
    activity=bd.Database(old_db_name).get(activity_code)
    new_act=copy.deepcopy(activity.as_dict())
    new_act['database']=new_db_name
    new_act['exchanges']=[] #empty list of exchanges to add add with for loop below
    new_act_key=(new_db_name,new_act['code']) #we just keep same code
#     new_act_key=(new_db_name,new_act['name']) #we put name for readibility

    down=None
    
    for exc in activity.exchanges():
        if exc['input'] not in polystyrene_upstream.values() or exc['type']=='production':
            data = copy.deepcopy(exc._data)
            data['output'] = new_act_key

            # Change `input` for production exchanges
            if exc['input'] == exc['output']:
                data['input'] = new_act_key        
            new_act['exchanges'].append(data)
        else:
            data = copy.deepcopy(exc._data)
            data['output'] = new_act_key
            #need to change the input 
            data['input'] = (new_db_name,exc['input'][1]) #if  exc['input'] not in polystyrene.values() else exc['input']
#             data['input'] = (new_db_name,exc['name']) #if  exc['input'] not in polystyrene.values() else exc['input']

            #add two years delay between materials raw materials(foam) and polystyrene productin
            if exc['input']==('ecoinvent-3.10-cutoff', '31c71922d2a8871fa590470a67c8193f'):
                data['temporal distribution']=[(-2,exc['amount'])] #remember python is 0 indexed so range is up to -1             

            new_act['exchanges'].append(data)
            down=data['input']
            
    return new_act,down


new_db_eco={}
key=polystyrene_act['polystyrene foam slab production']
while key != None:
    ds,key=change_polystyrene_act(key[1])
    
    new_db_eco[(ds['database'],ds['code'])]=ds

ok, we have copied the upstram exch now we just take all the tecnosphere exchanges of the copied EPS insulators production and spread them over the 100 years of rotation.

In [163]:
for i,exc in enumerate(new_db_eco[('time', 'de2b60a30ff7830518ec8873f647d26e')]['exchanges']):
    if exc['type']=='technosphere':
        new_db_eco[('time', 'de2b60a30ff7830518ec8873f647d26e')]['exchanges'][i]['temporal distribution']=[(0,exc['amount']/10)] 

EPS production and its EOL were modelleded separately in Excel and imported in a separate database (Insulators). Now we will take production and link it to the modified polystyrene with the relative temporal information (i.e. 2 years between polymer foam production and EPS). First we search for the dataset with EPS production and EOL. Then we search for the polystyrene exchange in the production dataset, modifies it with the one created above and add this modified dataset and the copy of the EOL dataset to the database we are creating

In [144]:
Insulators =  bd.Database('Insulators')

In [145]:
EPS=bd.Database('Insulators')
[[x['name'],x['code']] for x in EPS.search('EPS')]

[['EPS_EoL', '9503250025219c27273ebe1b28e0ac7a'],
 ['EPS_product', '10c1060a3a5c2d5bb0ea1149d9b8bf62']]

In [147]:
EPS_product=EPS.get('10c1060a3a5c2d5bb0ea1149d9b8bf62') #get EPS prod
EPS_EOL=EPS.get('10c1060a3a5c2d5bb0ea1149d9b8bf62') #get EPS eol

In [164]:
# for prod
new_EPS_code='EPS_product'

new_EPS=copy.deepcopy(EPS_product.as_dict())
new_EPS['database']=new_db_name
new_EPS['code']=new_EPS_code
new_EPS['name']=new_EPS_code
new_EPS['exchanges']=[] #empty list of exchanges to add add with for loop below
new_EPS_key=(new_db_name,new_EPS['code']) #we jsut keep same code and change db for simplicity

to_replace=(new_db_name,'de2b60a30ff7830518ec8873f647d26e')

for exc in EPS_product.exchanges():        
    data = copy.deepcopy(exc._data)
    data['output'] = new_EPS_key
    # Change `input` for production exchanges
    if exc['input'] == exc['output']:
        data['input'] = new_EPS_key
    #change sawnwood
    if exc['name']=='EPS_product':
        data['input']=to_replace
        data['name']=new_db_eco[to_replace]['name']
        data['temporal distribution']=[(-2,exc['amount'])]  # sawnwood is produced 2 years before being used by glulam 
    new_EPS['exchanges'].append(data)
new_db_eco[new_EPS_key]=new_EPS
# new_EPS['exchanges']

In [165]:
#for eol
new_EPS_eol_code='EPS_EOL'

new_EPS_eol=copy.deepcopy(EPS_EOL.as_dict())
new_EPS_eol['database']=new_db_name
new_EPS_eol['code']=new_EPS_eol_code
new_EPS_eol['name']=new_EPS_eol_code
new_EPS_eol['exchanges']=[] #empty list of exchanges to add add with for loop below
new_EPS_key_eol=(new_db_name,new_EPS_eol['code']) #we jsut keep same code and change db for semplicity

to_replace=(new_db_name,'de2b60a30ff7830518ec8873f647d26e')

for exc in EPS_EOL.exchanges():        
    data = copy.deepcopy(exc._data)
    data['output'] = new_EPS_key_eol
    # Change `input` for production exchanges
    if exc['input'] == exc['output']:
        data['input'] = new_EPS_key_eol
    new_EPS_eol['exchanges'].append(data)
new_db_eco[new_EPS_key_eol]=new_EPS_eol
# new_EPS_eol

We assume a mean lifetime for EPS of 50 years

Now we could modify the EPS_EOL making its exchanges dynamic, but this would be more complicate and also logically weirder than creating a new dataset, that is actually our Functional Unit, that has input exchange one square meter of EPS_Product and one m2 of EPS_EOL, this latter spread over time considering its EPS_decay. We create manually the FU dataset as below and write the new database. The unit m2 is represented as "unit".

In [167]:
code_FU='EPS_FU'
new_db_eco[(new_db_name,code_FU)]={
 'database':new_db_name,
 'name':code_FU,
 'code':code_FU,
 'unit':'unit',
 'exchanges': 
    [{
   'amount': 1.0,
   'input': (new_db_name, new_EPS_code),
   'name': new_EPS_eol_code,
   'type': 'technosphere',
   'unit': 'unit'
    },{
   'amount': 1.0,
   'input': (new_db_name, new_EPS_eol_code),
   'name': new_EPS_code,
   'temporal distribution': TemporalDistribution(
                    np.array([50], dtype='timedelta64[Y]'),  # Yearly time steps
                    np.array([1])
   ),
   'type': 'technosphere',
   'unit': 'unit'
    }],
}

time = bd.Database("time")
time.write(new_db_eco)

14:04:24 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 2538.31it/s]


14:04:24 [info     ] Vacuuming database            


OperationalError: cannot VACUUM - SQL statements in progress

In [169]:
for x in [  ('time', 'EPS_product'),
            ('time', 'EPS_EOL'),
            ]:
    act=bd.get_activity(x)
    act['time_group'] = True
    act.save()

UnknownObject: 

In [61]:
#select the activities to perform the LCA
#We want to compare the dynamic climate change impact of two insulators: fossil based (EPS, insulation) and biobased (Straw_based, insulation)
Straw_Insul =  bd.Database('Insulators')